In [ ]:
!pip install autogluon.tabular[all]

In [3]:
# import libraries
from autogluon.tabular import TabularDataset, TabularPredictor
from sklearn.model_selection import train_test_split
import pandas as pd

In [4]:
# load dataset
df = pd.read_csv('../input/zikavirusdataset/classification_dataset.csv')
df.head()

,density_per_km,latitude,longitude,maxtempC,mintempC,totalSnow_cm,sunHour,moon_illumination,DewPointC,WindGustKmph,cloudcover,humidity,precipMM,pressure,visibility,winddirDegree,year,month,Target
0,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,87,6,16,0,59,0.0,1018,10,189,2016,3,1
1,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,87,6,16,0,59,0.0,1018,10,189,2016,3,0
2,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,87,6,16,0,59,0.0,1018,10,189,2016,3,1
3,12625.800781,-34.603684,-58.381559,24,7,0.0,11.6,87,6,16,0,59,0.0,1018,10,189,2016,3,0
4,12625.800781,-34.603684,-58.381559,22,10,0.0,11.5,76,10,27,34,72,0.0,1025,10,99,2016,3,0


In [5]:
# Rows and cols
df.shape

(155223, 19)

In [6]:
# toal duplicate values
df.duplicated().sum()

79878

In [7]:
# Remove duplicate rows
df = df.drop_duplicates(keep='first', inplace=False)

In [9]:
df.shape

(75345, 19)

# Train Test Split

In [10]:
train_data,test_data = train_test_split(df,test_size=0.40,random_state=0,stratify=df['Target'] ) #train test split in 60:40 ratio

In [11]:
train_data.shape, test_data.shape

((45207, 19), (30138, 19))

In [12]:
train_data['Target'].value_counts(), test_data['Target'].value_counts()

(0    24440
 1    20767
 Name: Target, dtype: int64,
 0    16294
 1    13844
 Name: Target, dtype: int64)

# Build Model

In [13]:
label = 'Target'
metric = 'roc_auc'  # specify your evaluation metric here
predictor = TabularPredictor(label=label, eval_metric=metric).fit(train_data, presets='best_quality')

[1000]	train_set's binary_logloss: 0.196385	valid_set's binary_logloss: 0.271045
[1000]	train_set's binary_logloss: 0.195602	valid_set's binary_logloss: 0.284641
[2000]	train_set's binary_logloss: 0.145881	valid_set's binary_logloss: 0.282455
[1000]	train_set's binary_logloss: 0.19601	valid_set's binary_logloss: 0.269292
[2000]	train_set's binary_logloss: 0.145735	valid_set's binary_logloss: 0.268352
[1000]	train_set's binary_logloss: 0.197759	valid_set's binary_logloss: 0.259421
[2000]	train_set's binary_logloss: 0.148116	valid_set's binary_logloss: 0.257776
[1000]	train_set's binary_logloss: 0.196463	valid_set's binary_logloss: 0.278958
[1000]	train_set's binary_logloss: 0.197141	valid_set's binary_logloss: 0.269757
[1000]	train_set's binary_logloss: 0.196839	valid_set's binary_logloss: 0.270652
[2000]	train_set's binary_logloss: 0.147113	valid_set's binary_logloss: 0.268049
[1000]	train_set's binary_logloss: 0.1961	valid_set's binary_logloss: 0.27684
[2000]	train_set's binary_loglos

In [14]:
results = predictor.fit_summary(show_plot=True)

*** Summary of fit() ***
Estimated performance of each model:
                      model  score_val  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0       WeightedEnsemble_L3   0.949720      37.210031  3214.693246                0.017724          19.351125            3       True         26
1   RandomForestGini_BAG_L2   0.947713      25.892469  1979.724148                1.967376          19.276930            2       True         17
2   RandomForestEntr_BAG_L2   0.947576      25.834585  1989.950431                1.909492          29.503213            2       True         18
3     ExtraTreesGini_BAG_L2   0.947308      26.276035  1965.778201                2.350942           5.330984            2       True         20
4     ExtraTreesEntr_BAG_L2   0.947140      26.215806  1966.597646                2.290713           6.150428            2       True         21
5      LightGBMLarge_BAG_L2   0.946680      24.380373  2018.434395  

In [15]:
predictor.leaderboard(train_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,KNeighborsDist_BAG_L1,0.906159,0.249756,3.381042,3.423805,0.164611,3.381042,3.423805,0.164611,1,True,2
1,RandomForestEntr_BAG_L1,0.906159,0.383472,4.962452,2.755520,22.310024,4.962452,2.755520,22.310024,1,True,6
2,RandomForestGini_BAG_L1,0.906159,0.380056,3.824811,2.675128,14.287951,3.824811,2.675128,14.287951,1,True,5
3,ExtraTreesEntr_BAG_L1,0.906158,0.347511,6.283259,3.087351,8.128519,6.283259,3.087351,8.128519,1,True,9
4,ExtraTreesGini_BAG_L1,0.906158,0.346693,5.759523,3.122340,7.322661,5.759523,3.122340,7.322661,1,True,8
5,LightGBMLarge_BAG_L1,0.754929,0.662764,4.378022,0.228082,20.823881,4.378022,0.228082,20.823881,1,True,13
6,KNeighborsUnif_BAG_L1,0.741008,0.494399,3.482374,2.724805,0.194548,3.482374,2.724805,0.194548,1,True,1
7,XGBoost_BAG_L1,0.732267,0.656357,1.560439,0.198474,11.363792,1.560439,0.198474,11.363792,1,True,11
8,CatBoost_BAG_L1,0.731273,0.665181,0.320601,0.076926,103.492475,0.320601,0.076926,103.492475,1,True,7
9,LightGBM_BAG_L1,0.728489,0.671102,5.371019,0.234921,15.396821,5.371019,0.234921,15.396821,1,True,4


# Make Prediction

In [17]:
y_test = test_data[label]  # values to predict
test_data_nolab = test_data.drop(columns=[label])  # delete label column to prove we're not cheating
test_data_nolab.head()

,density_per_km,latitude,longitude,maxtempC,mintempC,totalSnow_cm,sunHour,moon_illumination,DewPointC,WindGustKmph,cloudcover,humidity,precipMM,pressure,visibility,winddirDegree,year,month
85860,49.728394,4.156630,-76.287762,27,16,0.0,12.5,59,14,5,18,68,16.6,1015,9,207,2016,9
50844,59.433132,9.144527,-74.226294,31,23,0.0,11.9,100,24,8,57,85,6.2,1010,7,229,2016,5
138749,209.804016,15.594288,-90.149499,30,17,0.0,13.0,84,17,7,13,74,1.0,1012,8,140,2016,5
107305,147.182251,19.052706,-70.149226,27,18,0.0,6.2,64,19,8,77,82,1.5,1016,7,158,2016,4
154932,835.046448,18.335765,-64.896335,28,27,0.0,8.7,96,23,41,42,77,12.9,1017,9,82,2016,12


In [18]:
predictor = TabularPredictor.load('./AutogluonModels/ag-20210921_175424')  # unnecessary, just demonstrates how to load previously-trained predictor from file

y_pred = predictor.predict(test_data_nolab)
print("Predictions:  \n", y_pred)

Predictions:  
 85860     0
50844     1
138749    1
107305    0
154932    1
         ..
76754     1
50884     1
141791    0
22127     1
60368     0
Name: Target, Length: 30138, dtype: int64


In [20]:
predictor.leaderboard(test_data, silent=True)

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,NeuralNetFastAI_BAG_L2,0.944730,0.944223,47.756374,25.167060,2863.852661,7.905569,1.241967,903.405443,2,True,22
1,LightGBM_BAG_L2,0.944463,0.946023,42.513519,24.348277,1994.704804,2.662714,0.423184,34.257587,2,True,16
2,CatBoost_BAG_L2,0.944335,0.944689,40.058973,24.021507,2072.585986,0.208168,0.096414,112.138768,2,True,19
3,LightGBMLarge_BAG_L2,0.944282,0.946680,42.661689,24.380373,2018.434395,2.810885,0.455280,57.987177,2,True,25
4,WeightedEnsemble_L3,0.943865,0.949720,61.983417,37.210031,3214.693246,0.008039,0.017724,19.351125,3,True,26
5,XGBoost_BAG_L2,0.943855,0.944776,41.619082,24.199243,1993.609966,1.768278,0.274150,33.162748,2,True,23
6,RandomForestEntr_BAG_L2,0.943129,0.947576,40.788802,25.834585,1989.950431,0.937998,1.909492,29.503213,2,True,18
7,RandomForestGini_BAG_L2,0.942920,0.947713,40.828299,25.892469,1979.724148,0.977494,1.967376,19.276930,2,True,17
8,LightGBMXT_BAG_L2,0.942862,0.945452,58.892951,26.662084,2088.459737,19.042147,2.736991,128.012519,2,True,15
9,NeuralNetMXNet_BAG_L2,0.942005,0.942457,52.116419,27.795319,3042.835804,12.265615,3.870226,1082.388586,2,True,24


In [22]:
pred_probs = predictor.predict_proba(test_data_nolab)
pred_probs.head(5)

,0,1
85860,0.562518,0.437483
50844,0.000357,0.999643
138749,0.047992,0.952008
107305,0.999867,0.000132
154932,0.014611,0.985389


In [23]:
from sklearn.metrics import accuracy_score
y_predi = predictor.predict(test_data)
score = accuracy_score(y_test,y_predi)
print(score)

0.8397703895414427


In [26]:
# Accuracy with NeuralNetFastAI_BAG_L2
from sklearn.metrics import accuracy_score
y_predi = predictor.predict(test_data, model='NeuralNetFastAI_BAG_L2')
score = accuracy_score(y_test,y_predi)
print(score)

0.8406330877961378


In [25]:
# Accuracy with LightGBM_BAG_L2
from sklearn.metrics import accuracy_score
y_predi = predictor.predict(test_data, model='LightGBM_BAG_L2')
score = accuracy_score(y_test,y_predi)
print(score)

0.8401353772645829
